In [16]:
import matplotlib.pyplot as plt
import sys
sys.path.append('../main/')
from multi_py.multi import *
import numpy as np
import sys  
from os.path import dirname, join as pjoin
import scipy.io as sio
from scipy.io import readsav
import copy
import pandas as pd
import os
from scipy import integrate
from scipy.signal import argrelextrema
from scipy import interpolate
import pickle

%config InlineBackend.figure_format='png'
plt.style.use('default')
plt.style.use('../main/multi_py/Bergemann2020.mplstyle')

## Read in observed spectrum

In [17]:
sav_fname = pjoin('../../data/spectra/1631885174310_iag_sun_Si.sav')
sav_data = readsav(sav_fname)
# f = open('../../data/spectra/sun_kpnomu1_6000_8000A.txt')
# obs_spec2 = np.loadtxt(fname=f)
# f.close()

In [18]:
# simulation has been run with mus: 1, 0.9, 0.8, 0.6, 0.4, 0.2
# specify the indecies corresponding to these angles in the solar data
sol_mu_ind = [0, 4, 5, 7, 9, 11]
# get the flux(intensity) and wavelengths corresponding to these angles
sol_fluxes = [sav_data['iag_sun'].flux[0][ang] for ang in sol_mu_ind]
sol_lams = [sav_data['iag_sun'].wave[0][ang] for ang in sol_mu_ind]
sol_mus = [sav_data['iag_sun'].mu[0][ang] for ang in sol_mu_ind]

major_krs = [286, 291, 293, 299, 427] # 292 and 302 removed - empty gaps; 457 removed - does not appear in the ref paper
m3d_centre_wavelengths = [5645.613, 5684.484, 5701.104, 5772.146, 7034.901]

## Calculate EWs for the observed spectra

In [19]:
# define the integration limits for the iag observed solar spectrum for each line (keep same across angles)
iag_int_lims = np.array([[5645.4, 5645.8],[5684.31,5684.7],[5700.97,5701.3],[5771.9,5772.35],[7034.55,7035.2]])

In [20]:
eq_iag = np.zeros((len(major_krs), len(sol_mu_ind)))
sol_lam_range_list = np.zeros((len(major_krs), len(sol_mu_ind)), dtype=object)
sol_int_range_list = np.zeros((len(major_krs), len(sol_mu_ind)), dtype=object)
for i, line in enumerate(major_krs):
    line_lammin, line_lammax = iag_int_lims[i]
    for j, mu in enumerate(sol_mu_ind):
        lam_range = sol_lams[j][np.logical_and((sol_lams[j] > line_lammin), (sol_lams[j] < line_lammax))]
        intensity_range = sol_fluxes[j][np.logical_and((sol_lams[j] > line_lammin), (sol_lams[j] < line_lammax))]
        eq_iag[i,j] = integrate.simps(y=1-intensity_range, x=lam_range) * 1000
        # record the wavelengths and intensities corresponding to each line
        sol_lam_range_list[i,j] = lam_range
        sol_int_range_list[i,j] = intensity_range

## Plot zoomed-out observed spectra 

In [21]:
## Plot zoomed-out observed spectra 
# fig, ax = plt.subplots(len(major_krs), 1, figsize=(15,20))

# for i, thiskr in enumerate(major_krs):
    
#     for j, thismu in enumerate(sol_mus):
#         # plt.sca(ax[j])
#         ax[i].plot(sol_lams[j], sol_fluxes[j], zorder=100, label=sol_mus[j])#,markersize=4)


#     # plot picked integration limits 
#     for lim in iag_int_lims[i,:]:
#         ax[i].axvline(lim)
#     ax[i].grid(axis='y')

#     ax[i].set_title(f'Si {wavelengths[i]:.2f} Å')
#     ax[i].set_ylabel('Norm. Intensity')
#     ax[i].set_xlabel('λ, Å')

#     plt.sca(ax[i])

#     plt.xlim((m3d_centre_wavelengths[i]-3), (m3d_centre_wavelengths[i]+3))
#     plt.ylim(0.4,1.1)    
#     ax[i].legend()

# plt.tight_layout()

## Deblending, renormalising lines in observed spectra

In [22]:
from scipy.signal import argrelextrema
from scipy.special import voigt_profile
from scipy import optimize

def synth_blends(obs_xx, obs_yy, order=20, blank_lam=None):
    exts = np.array(argrelextrema(obs_yy, np.less, order=order))[0]
    
    pn=5
    def model_func(xdata, *args):
        yy_pred = np.ones_like(xdata)
        
        for i, gamma in enumerate(args[::pn]):
            sigma = args[1::pn][i]
            scale = args[2::pn][i]
            shift = args[3::pn][i]
            skew  = args[4::pn][i]
            ext = exts[i]
            
            sxx = xdata - xdata[ext] + shift
            voigt = voigt_profile(sxx, sigma, gamma)
            
            sk_xx = sxx+np.abs(sxx)*skew
            sk_voigt = np.interp(sxx, sk_xx, voigt)
            
            yy_pred -= sk_voigt / np.max(sk_voigt) * (1-obs_yy[ext]) * scale
            
        return yy_pred
    
    params = np.ones(exts.size*pn)*1e-2
    params[2::pn] = 1
    params[3::pn] = 0
    params[4::pn] = 0
    
    pfit, pcov = optimize.curve_fit(model_func, obs_xx, obs_yy, p0=params)
    
    if blank_lam is not None:
        pfit_blank = pfit.copy()
        iline = np.where(obs_xx[exts] > blank_lam)[0][0]
        pfit_blank[pn*iline+2] = 0
        
        yy = model_func(obs_xx, *pfit_blank)
        
    else:
        yy = model_func(obs_xx, pfit)
    
    return yy

In [23]:
line_0_eqw = []
line_0_int = []
for j, mu in enumerate(sol_mus):
    xx = sol_lams[j]
    yy = sol_fluxes[j]
    lam0 = m3d_centre_wavelengths[0]
    mask = np.abs(xx - lam0) < 0.4
    xx = xx[mask]
    yy = yy[mask]
    

    thislam0 = xx[np.argmin(yy)]
    yy_blend = synth_blends(xx, yy, order=25, blank_lam=0)
    # yy_blend = synth_blends(xx, yy, order=20, blank_lam=lam0)
    yy_deblend = 1 - ((1-yy) - (1-yy_blend))

    line_0_eqw.append(integrate.simps(y=1-yy_deblend, x=xx) * 1000)
    line_0_int.append(yy_deblend)

    # overwrite lambda/int range for this line 
    sol_lam_range_list[0,j] = xx
    sol_int_range_list[0,j] = yy_deblend

In [24]:
# sim_int_1d = get_ang_intensity(runobjects[0], kr=major_krs[line_ind])
# sim_int_1d_norm = sim_int_1d/continuum(line_lam, np.array(sim_int_1d))

# define continuum at the chosen integration limits for each line
# renorm_lines_indx = [3,4]
sol_int_renorm = np.zeros((len(major_krs), len(sol_mu_ind)), dtype=object)

for i, line in enumerate(major_krs):
    for j, mu in enumerate(sol_mus):
        sol_int_renorm[i,j] = sol_int_range_list[i,j]/continuum(sol_lam_range_list[i,j], sol_int_range_list[i,j])

In [25]:
# replot with new normalisation
plot=False
if plot==True:
    fig, ax = plt.subplots(len(major_krs), 1, figsize=(13,20))

    for i, line_i in enumerate(major_krs):
        
        for j, thismu in enumerate(sol_mus):
            # plt.sca(ax[j])
            ax[i].plot(sol_lam_range_list[i,j], sol_int_renorm[i,j], zorder=100, label=sol_mus[j],markersize=4)
            # ax[i].plot(sol_lam_range_list[i,j], sol_int_range_list[i,j], linestyle='none', marker='.', zorder=100, label=sol_mus[j],markersize=4)


        # plot picked integration limits 
        for lim in iag_int_lims[i,:]:
            ax[i].axvline(lim)
        ax[i].grid(axis='y')

        ax[i].set_title(f'Si {wavelengths[i]:.2f} Å')
        ax[i].set_ylabel('Norm. Intensity')
        ax[i].set_xlabel('λ, Å')

        plt.sca(ax[i])

        plt.xlim((m3d_centre_wavelengths[i]-0.25), (m3d_centre_wavelengths[i]+0.25))
        plt.ylim(0.4,1.1)    
        ax[i].legend()

    plt.tight_layout()

In [26]:
sol_mus

[1.0, 0.9, 0.8, 0.6, 0.4, 0.2]

In [27]:
# recalculate equivalent widths
new_eq_iag = np.zeros(eq_iag.shape)
for i, line_i in enumerate(major_krs):
    for j, thismu in enumerate(sol_mus):
            new_eq_iag[i,j] = integrate.simps(y=1-sol_int_renorm[i,j], x=sol_lam_range_list[i,j]) * 1000

In [31]:
with open('../mdata/new_eq_iag', 'wb') as f:
    pickle.dump(new_eq_iag, f)
# outfile = open('./mdata/new_eq_iag')
# pickle.dump(new_eq_iag, open('./mdata/new_eq_iag', 'wb'))

In [32]:
"\N{Sauropod}"

'🦕'